# Traning thêm data sentence bert dùng word2vec để tính similarity of sentence

## Các bước thực hiện:

1. Load data từ database, dùng underthesea để tách câu.
2. Dùng word2vec để tính vector của câu.
3. Training data cho sentence bert dùng vector sinh ra ở [2].



In [3]:
from sklearn.feature_extraction import _stop_words
import string
from tqdm.autonotebook import tqdm
import numpy as np
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch
import pickle
import pandas as pd

# how to import module from ../alorithm/
import sys
sys.path.append('../algorithm')
from importlib import reload
import helper
import regress_adapter
reload(helper)
reload(regress_adapter)


<module 'regress_adapter' from '/Users/ngocp/Documents/projects/pyml/botapp/research/../algorithm/regress_adapter.py'>

In [4]:

data = regress_adapter.get_data('TPB', '2022-01-01', '2023-09-29')
print(data.head())

/Users/ngocp/Documents/projects/pyml/botapp/research/../algorithm/regress_adapter.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


                                                   text  close      date_y
3790  VNDIRECT: Hầu hết giá cổ phiếu của dệt may hiệ...  27680  2022-01-04
3789  Cách tốt nhất để các nhà đầu tư bảo vệ mình kh...  27680  2022-01-04
3788  Đảng uỷ cơ quan Uỷ ban Chứng khoán Nhà nước: S...  27680  2022-01-04
3787  Truyền hình K+ tặng đầu thu miễn phí đón ‘Tết ...  27680  2022-01-04
3786  Licogi 14 (L14) “bơm” thêm 214 tỷ đồng đầu tư ...  27190  2022-01-05


## First we build model word2vec to caculate similarity of sentence

In [6]:
data_list = data['text'].values.tolist()
print(data_list[:2])

def split_word_token(texts = []):
    clear_ld = lambda t: helper.NLP(t).get_words_feature()
    t = map(clear_ld, texts)
    return list(t)

data_token = split_word_token(data_list)

['VNDIRECT: Hầu hết giá cổ phiếu của dệt may hiện đang gần với giá trị hợp lý, động lực tăng trưởng đến từ nhiều dự án khu công nghiệp\n\n\n\n\r\n                    TNG&MSH&STK&ADS&GIL&TCM&VGT: \n\n\n\n\nGiá hiện tại\n\n\n\n\nThay đổi\n\n\n\n\n\n\n\nXem hồ sơ doanh nghiệp \n\n\n\n\nTIN MỚI\n\n\n\n\nNhiều mã cổ phiếu dệt may tăng mạnhTheo phân tích mới đây từ Trung tâm Thông tin công nghiệp và thương mại (Bộ Công thương), năm 2021, ngành dệt may Việt Nam đã về đích với 39 tỷ USD kim ngạch xuất khẩu, tăng 11,2% so với năm 2020, tương đương với thời điểm trước dịch (năm 2019). Nhiều doanh nghiệp dệt may có KQKD tăng trưởng ấn tượng: Theo ước tính của VNDIRECT, tổng doanh thu Q4/21 của các công ty dệt may niêm yết tăng 24,1% svck, trong khi LN ròng của ngành tăng 57,0% svck trong Q4/21, cao hơn 82,0% so với Q3/21. Các doanh nghiệp có mức tăng trưởng LN ấn tượng phải kể đến ADS (+ 303,2% svck), VGT (+ 161,4% svck) và STK (+ 92,9% svck).So với thời điểm dịch 

In [7]:
from gensim.models import Word2Vec

model = Word2Vec(data_token, vector_size=100, epochs=100, workers=4)

FileNotFoundError: [Errno 2] No such file or directory: '../cached/sBertTrain-word2vec_new.model'

In [8]:
model.save('./cached/sBertTrain-word2vec_new.model')

In [10]:
def split_sents(texts = []):
    def split_sentences(text):
        try:
            sents = helper.NLP(text).split_sentences()
        except Exception as e:
            print('Error:', e, text)
            sents = []
        sents_1 = [sent for sent in sents if sent.strip() != ""]
        return sents_1

    t = map(split_sentences, texts)
    return list(t)

def flatten_list(l):
    return [item for sublist in l for item in sublist]

In [11]:
sents = split_sents(data_list)

Error: expected string or bytes-like object nan
Error: expected string or bytes-like object nan
Error: expected string or bytes-like object nan
Error: expected string or bytes-like object nan
Error: expected string or bytes-like object nan
Error: expected string or bytes-like object nan
Error: expected string or bytes-like object nan
Error: expected string or bytes-like object nan
Error: expected string or bytes-like object nan
Error: expected string or bytes-like object nan
Error: expected string or bytes-like object nan


In [12]:
print(len(sents))

5085


In [16]:
# Calculate sentence embeddings by averaging word embeddings
def calculate_sentence_embedding(embeddings):
    if len(embeddings) == 0:
        return np.zeros(model.vector_size)
    return np.mean(embeddings, axis=0)

def get_embed_sent(sent):
    sent_words = split_word_token([sent])
    question_embeddings = [model.wv[word] for word in sent_words[0] if word in model.wv]
    question_embedding = calculate_sentence_embedding(question_embeddings)
    return question_embedding

print('sent:', sents[0][0])
t = get_embed_sent(sents[0][0])
print(t.shape)

sent: VNDIRECT: Hầu hết giá cổ phiếu của dệt may hiện đang gần với giá trị hợp lý, động lực tăng trưởng đến từ nhiều dự án khu công nghiệp




                    TNG&MSH&STK&ADS&GIL&TCM&VGT: 




Giá hiện tại




Thay đổi







Xem hồ sơ doanh nghiệp 




TIN MỚI




Nhiều mã cổ phiếu dệt may tăng mạnhTheo phân tích mới đây từ Trung tâm Thông tin công nghiệp và thương mại (Bộ Công thương), năm 2021, ngành dệt may Việt Nam đã về đích với 39 tỷ USD kim ngạch xuất khẩu, tăng 11,2% so với năm 2020, tương đương với thời điểm trước dịch (năm 2019).
(100,)


In [19]:
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses
from sklearn.metrics.pairwise import cosine_similarity

inputExpTrain = []
for post in sents:
    for i, sentence in enumerate(post):
        if i == len(post) - 1:
            break
        sent1 = sentence
        sent2 = post[i+1]
        label = cosine_similarity([get_embed_sent(sent1)], [get_embed_sent(sent2)])
        inputExpTrain.append([sentence, post[i+1], label[0][0]])
       

# construct dataframe from inputExpTrain
df = pd.DataFrame(inputExpTrain, columns=['text1', 'text2', 'label'])


In [20]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109810 entries, 0 to 109809
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   text1   109810 non-null  object 
 1   text2   109810 non-null  object 
 2   label   109810 non-null  float64
dtypes: float64(1), object(2)
memory usage: 2.5+ MB
None


In [21]:
# how to min max scale for column label in df
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
df['label'] = min_max_scaler.fit_transform(df['label'].values.reshape(-1,1))
print(df.head())

                                               text1  \
0  VNDIRECT: Hầu hết giá cổ phiếu của dệt may hiệ...   
1  Nhiều doanh nghiệp dệt may có KQKD tăng trưởng...   
2  Các doanh nghiệp có mức tăng trưởng LN ...   
3  Xuất khẩu vải và hàng may mặc trong Quý IV/202...   
4  Nguồn: MOIT, VNDIRECT ResearchHưởng lợi từ chi...   

                                               text2     label  
0  Nhiều doanh nghiệp dệt may có KQKD tăng trưởng...  0.802747  
1  Các doanh nghiệp có mức tăng trưởng LN ...  0.763249  
2  Xuất khẩu vải và hàng may mặc trong Quý IV/202...  0.761638  
3  Nguồn: MOIT, VNDIRECT ResearchHưởng lợi từ chi...  0.674743  
4  Tháng 12/2021, tổng thống Joe Biden ký sắc lện...  0.782528  


In [23]:
# load df into list of InputExample
data_train = []
for index, row in df.iterrows():
    data_train.append(InputExample(texts=[row['text1'], row['text2']], label=row['label']))

In [ ]:
df.to_csv('./data/sentences-train.csv', index=False)

In [24]:
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses
from torch.utils.data import DataLoader

# Load the pre-trained model
model = SentenceTransformer('nli-distilroberta-base-v2')

#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(data_train, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=10, warmup_steps=100)


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/6864 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:12                                                                                   │
│                                                                                                  │
│    9 train_loss = losses.CosineSimilarityLoss(model)                                             │
│   10                                                                                             │
│   11 #Tune the model                                                                             │
│ ❱ 12 model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=10, warmup_steps=100    │
│   13                                                                                             │
│                                                                                                  │
│ /Users/ngocp/.pyenv/versions/3.10.3/lib/python3.10/site-packages/sentence_transformers/SentenceT │
│ ransformer.py:722 in fit                                                                         │
│                                                                                                  │
│   719 │   │   │   │   │   │   skip_scheduler = scaler.get_scale() != scale_before_step           │
│   720 │   │   │   │   │   else:                                                                  │
│   721 │   │   │   │   │   │   loss_value = loss_model(features, labels)                          │
│ ❱ 722 │   │   │   │   │   │   loss_value.backward()                                              │
│   723 │   │   │   │   │   │   torch.nn.utils.clip_grad_norm_(loss_model.parameters(), max_grad   │
│   724 │   │   │   │   │   │   optimizer.step()                                                   │
│   725                                                                                            │
│                                                                                                  │
│ /Users/ngocp/.pyenv/versions/3.10.3/lib/python3.10/site-packages/torch/_tensor.py:487 in         │
│ backward                                                                                         │
│                                                                                                  │
│    484 │   │   │   │   create_graph=create_graph,                                                │
│    485 │   │   │   │   inputs=inputs,                                                            │
│    486 │   │   │   )                                                                             │
│ ❱  487 │   │   torch.autograd.backward(                                                          │
│    488 │   │   │   self, gradient, retain_graph, create_graph, inputs=inputs                     │
│    489 │   │   )                                                                                 │
│    490                                                                                           │
│                                                                                                  │
│ /Users/ngocp/.pyenv/versions/3.10.3/lib/python3.10/site-packages/torch/autograd/__init__.py:200  │
│ in backward                                                                                      │
│                                                                                                  │
│   197 │   # The reason we repeat same the comment below is that                                  │
│   198 │   # some Python versions print out the first line of a multi-line function               │
│   199 │   # calls in the traceback and some print out the last line                              │
│ ❱ 200 │   Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the bac   │
│   201 │   │   tensors, grad_tensors_, retain_graph, create_graph, inputs,                        │
│   202 │   │   allow_unreachable=True, accumulate_grad=True)